In [6]:
import csv # easier to read two rows at a time vs pandas
import pandas as pd

In [7]:
# Create list to store games
game_list = list()

# Create list to store games for 2024 (no true labels in the dataset)
games_without_label = list()
# Create dictionary to hold all Team Data
teams_dict = dict()

In [8]:
"""
Load all pickle files
"""

import pickle

# Loading season team data
with open('pickle_files/538-season-team-data.pkl', 'rb') as season_team_file:
    season_team_data = pickle.load(season_team_file)
    
# Loading conference data
with open('pickle_files/conference-data.pkl', 'rb') as conference_file:
    conference_data = pickle.load(conference_file)

# Loading kenpom/barttorvik data
with open('pickle_files/kenpom-barttorvik-combined.pkl', 'rb') as kenpom_file:
    kenpom_data = pickle.load(kenpom_file)

# Loading resume data
with open('pickle_files/resume-data.pkl', 'rb') as resume_file:
    resume_data = pickle.load(resume_file)

# Loading kenpom/barttorvik data
with open('pickle_files/seed-full-data.pkl', 'rb') as seed_file:
    seed_data = pickle.load(seed_file)
    
# Loading kenpom/barttorvik data
with open('pickle_files/shooting-splits-data.pkl', 'rb') as shooting_splits_file:
    shooting_splits_data = pickle.load(shooting_splits_file)

# Loading historical team data
with open('pickle_files/538-historic-team-data.pkl', 'rb') as historical_team_file:
    historical_team_data = pickle.load(historical_team_file)
    
# Loading conference mapping
with open('pickle_files/team-conference-mapping.pkl', 'rb') as conference_mapping_file:
    team_conference_mapping = pickle.load(conference_mapping_file)

In [9]:
# DEBUG CHECK

print(shooting_splits_data[("Houston", 2023)], '\n\n')
print(seed_data, '\n\n')
print(resume_data[("Houston", 2023)], '\n\n')
print(kenpom_data[("Houston", 2023)], '\n\n')
print(conference_data[("ACC", 2023)], '\n\n')
print(season_team_data[("Houston", 2023)])
print(historical_team_data["Houston"])

print("\n\n\n KEYS \n")

print(list(shooting_splits_data[("Houston", 2023)].keys()))
print(list(kenpom_data[("Houston", 2023)].keys()))
print(list(season_team_data[("Houston", 2023)].keys()))
print(list(resume_data[("Houston", 2023)].keys()))
print(list(historical_team_data["Houston"].keys()))

{'(Shooting Splits) DUNKS FG%': 94.0, '(Shooting Splits) DUNKS SHARE': 9.8, '(Shooting Splits) DUNKS FG%D': 83.6, '(Shooting Splits) DUNKS D SHARE': 6.1, '(Shooting Splits) CLOSE TWOS FG%': 67.4, '(Shooting Splits) CLOSE TWOS SHARE': 29.7, '(Shooting Splits) CLOSE TWOS FG%D': 55.1, '(Shooting Splits) CLOSE TWOS D SHARE': 32.2, '(Shooting Splits) FARTHER TWOS FG%': 39.9, '(Shooting Splits) FARTHER TWOS SHARE': 32.4, '(Shooting Splits) FARTHER TWOS FG%D': 27.8, '(Shooting Splits) FARTHER TWOS D SHARE': 24.4, '(Shooting Splits) THREES FG%': 34.0, '(Shooting Splits) THREES SHARE': 37.9, '(Shooting Splits) THREES FG%D': 27.9, '(Shooting Splits) THREES D SHARE': 43.5} 


{1: <Seed.Seed object at 0x116d04880>, 11: <Seed.Seed object at 0x116d04850>, 3: <Seed.Seed object at 0x116d047f0>, 15: <Seed.Seed object at 0x116d04790>, 7: <Seed.Seed object at 0x116d04730>, 12: <Seed.Seed object at 0x116d046d0>, 8: <Seed.Seed object at 0x116d04670>, 4: <Seed.Seed object at 0x116d04610>, 16: <Seed.Seed obj

In [16]:
from Conference import Conference
from Game import Game
from Seed import Seed
from Team import Team


with open("MM Data/Tournament Matchups.csv", 'r') as game_file:
    reader = csv.DictReader(game_file)
    
    # Need to read two at a time, so we will just append to this list and when there are 2 elements, process and set it back to empty
    temp_rows = []
    
    for row in reader:
        temp_rows.append(row)
        
        # Process when ""full""
        if len(temp_rows) == 2:
            
            first_team_row, second_team_row = temp_rows[0], temp_rows[1]
            
            first_year, first_team_name, first_seed = int(first_team_row["YEAR"]), first_team_row["TEAM"].strip(), int(first_team_row["SEED"])
            first_conf = team_conference_mapping[first_team_name]
            
            second_year, second_team_name, second_seed = int(second_team_row["YEAR"]), second_team_row["TEAM"].strip(), int(second_team_row["SEED"])
            second_conf = team_conference_mapping[second_team_name]
            
            
            # Check if first team already exists in the dictionary for this year, otherwise create the team object
            if (first_team_name, first_year) not in teams_dict:
                teams_dict[(first_team_name, first_year)] = Team(
                    team_name=first_team_name,
                    seed=seed_data[first_seed], # This works because I know the dataset is complete, for the rest, I will need to provide default values
                    conference=conference_data.get((first_conf, first_year), Conference("None",{})),
                    year=first_year,
                    team_stats_538=season_team_data.get((first_team_name, first_year),{}),
                    resume_stats=resume_data.get((first_team_name, first_year),{}),
                    shooting_splits_stats=shooting_splits_data.get((first_team_name, first_year),{}),
                    kenpom_stats=kenpom_data.get((first_team_name, first_year),{}),
                    historic_team_stats=historical_team_data.get((first_team_name, first_year),{}),
                    
                )
                
            # Check if second team already exists in the dictionary for this year, otherwise create the team object
            if (second_team_name, second_year) not in teams_dict:
                teams_dict[(second_team_name, second_year)] = Team(
                    team_name=second_team_name,
                    seed=seed_data[second_seed], # This works because I know the dataset is complete, for the rest, I will need to provide default values
                    conference=conference_data.get((second_conf, second_year), Conference("None",{})),
                    year=second_year,
                    team_stats_538=season_team_data.get((second_team_name, second_year),{}),
                    resume_stats=resume_data.get((second_team_name, second_year),{}),
                    shooting_splits_stats=shooting_splits_data.get((second_team_name, second_year),{}),
                    kenpom_stats=kenpom_data.get((second_team_name, second_year),{}),
                    historic_team_stats=historical_team_data.get((second_team_name, second_year),{}),
                    
                )
            
            # Create Game (if 2024 add it to the unlabeled pile)
            
            if first_year == 2024:
                games_without_label.append(
                    Game(
                        year=first_year,
                        team1=teams_dict[(first_team_name, first_year)],
                        team2=teams_dict[(second_team_name, second_year)],
                        score_differential=None
                    )
                )
            else:
                score_differential = int(first_team_row["SCORE"]) - int(second_team_row["SCORE"])
                game_list.append(
                    Game(
                        year=first_year,
                        team1=teams_dict[(first_team_name, first_year)],
                        team2=teams_dict[(second_team_name, second_year)],
                        score_differential=score_differential
                    )
                )
                
            temp_rows = []  # Reset the temp
            
            # A
            
            

In [17]:
len(teams_dict)

1027

In [18]:
teams_dict

{('Connecticut', 2024): <Team.Team at 0x1043c1870>,
 ('Stetson', 2024): <Team.Team at 0x117583490>,
 ('North Carolina', 2024): <Team.Team at 0x117581ae0>,
 ('Howard', 2024): <Team.Team at 0x116a70c40>,
 ('Wagner', 2024): <Team.Team at 0x116a71930>,
 ('Houston', 2024): <Team.Team at 0x116a71a20>,
 ('Longwood', 2024): <Team.Team at 0x116a719c0>,
 ('Purdue', 2024): <Team.Team at 0x116a71630>,
 ('Montana St.', 2024): <Team.Team at 0x116a71c90>,
 ('Grambling St.', 2024): <Team.Team at 0x116a71b40>,
 ('Iowa St.', 2024): <Team.Team at 0x116a71b70>,
 ('South Dakota St.', 2024): <Team.Team at 0x116a72950>,
 ('Arizona', 2024): <Team.Team at 0x116a728f0>,
 ('Long Beach St.', 2024): <Team.Team at 0x116a72890>,
 ('Marquette', 2024): <Team.Team at 0x116a71030>,
 ('Western Kentucky', 2024): <Team.Team at 0x116a70040>,
 ('Tennessee', 2024): <Team.Team at 0x116a72b60>,
 ("Saint Peter's", 2024): <Team.Team at 0x116a72bc0>,
 ('Illinois', 2024): <Team.Team at 0x116a71150>,
 ('Morehead St.', 2024): <Team.T

In [19]:
len(game_list), len(games_without_label)

(944, 74)

In [20]:
# Dump game data
with open('final_data_pickles/games-with-scores.pkl', 'wb') as file:
    pickle.dump(game_list, file, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('final_data_pickles/games-without-scores.pkl', 'wb') as file:
    pickle.dump(games_without_label, file, protocol=pickle.HIGHEST_PROTOCOL)
    
# Dump teams data as well
with open('final_data_pickles/full-team-data.pkl', 'wb') as full_team_file:
    pickle.dump(teams_dict, full_team_file, protocol=pickle.HIGHEST_PROTOCOL)